# Исследование надёжности заёмщиков — анализ банковских данных

## Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. 

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Цель исследования — проверить гипотезы:

- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?


## Описание данных

Ниже описание колонок из таблицы `/datasets/data.csv`:

- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

## 1. Изучим общую информацию

In [1]:
# Импортируем библиотеку pandas
import pandas as pd

# Считываем csv-файл с исходными данными
data = pd.read_csv('datasets/data.csv')
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [2]:
# Выводим общую информацию о данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# Выводим количество значений в колонке 'children'
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [4]:
# Выводим отсортированные по возрастанию значения в колонке 'days_employed'
data['days_employed'].sort_values()

16335   -18388.949901
4299    -17615.563266
7329    -16593.472817
17838   -16264.699501
16825   -16119.687737
             ...     
21489             NaN
21495             NaN
21497             NaN
21502             NaN
21510             NaN
Name: days_employed, Length: 21525, dtype: float64

In [5]:
# Выводим значения в колонке 'days_employed', которые больше 0
data[data['days_employed'] > 0]['days_employed'].sort_values().tail(10)

8369     401590.452231
10991    401591.828457
17823    401614.475622
13420    401619.633298
4697     401635.032697
7794     401663.850046
2156     401674.466633
7664     401675.093434
10006    401715.811749
6954     401755.400475
Name: days_employed, dtype: float64

In [6]:
# Выводим отсортированные по возрастанию уникальные значения в колонке 'dob_years'
data['dob_years'].sort_values(ascending=True).unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

In [7]:
# Выводим количество значений в колонке 'education'
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [8]:
# Выводим количество значений в колонке 'education_id'
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [9]:
# Выводим количество значений в колонке 'family_status'
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [10]:
# Выводим количество значений в колонке 'family_status_id'
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [11]:
# Выводим количество значений в колонке 'gender'
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [12]:
# Выводим количество значений в колонке 'income_type'
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [13]:
# Выводим количество значений в колонке 'debt'
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

In [14]:
# Выводим отсортированные по возрастанию значения в колонке 'total_income'
data['total_income'].sort_values()

14585    20667.263793
13006    21205.280566
16174    21367.648356
1598     21695.101789
14276    21895.614355
             ...     
21489             NaN
21495             NaN
21497             NaN
21502             NaN
21510             NaN
Name: total_income, Length: 21525, dtype: float64

In [15]:
# Выводим отсортированные по возрастанию значения в колонке 'total_income', которые меньше, либо равны 0
data[data['total_income'] <= 0]['total_income'].sort_values()

Series([], Name: total_income, dtype: float64)

In [16]:
# Выводим количество значений в колонке 'purpose'
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**Вывод**

Предварительно изучив полученные данные можно выделить следующие ошибки и проблемы:   
- Имеются пропуски данных в колонках `days_employed` и `total_income`
- В колонке `children` имеются отрицательные значения, что противоречит смыслу колонки
- Значения в колонке `days_employed` имеют как отрицательные, так и положительные значения, так же они не целочисленные, что противоречит смыслу колонки.
- В колонке `dob_years` имеются значения равные нулю. Но возраст не может быть равен нулю, в связи с чем может потребоваться корректировка данных значений или же их исключение из анализа
- Необходимо форматирование значений в колонке `education` для избавления от дублей
- Неоходимо создать словари для колонок `education` и `family_status`
- В колонке `total_income` может потребоватьсся группировка данных
- Колонка `purpose` требует лемматизации для последующей группировки данных по общим категориям

## 2. Предобработка данных

### Обработка пропусков

Нам уже известно, что в колонке `days_employed` имеются пропуски.<br>
Пропуски могли возникнуть по нескольким причинам:

- данные о клиенте отсутствуют
- клиент не захотел указывать информацию
- данные не подгрузились в отчет по техническим причинам
    

Попробуем найти какую-то закономерность в пропусках.<br>
Изучим взаимосвязь между возрастом клиента и пропуском в стаже работы. Возможно, пропущены значения для клиентов, не имеющих стажа работы.

In [17]:
# Создадим перемнную для таблицы с пропусками в колонке 'days_employed'
data_days_empty = data[data['days_employed'].isna()]

# Посомтрим клиенты каких возрастов попали в отфильтрованную таблицу
data_days_empty['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73])

Результаты говорят о том, что взаимосвязи между возрастом и пропусками нет, т.к. практически среди всех возрастов попадается данная проблема.<br><br>
Попробуем провести сравнение по колонке `income_type`.

In [18]:
# Посомтрим клиенты с какими должностями попали в отфильтрованную таблицу
data_days_empty['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

В данном случае тоже не видно сильной взаимосвязи. К тому же мы видим, что причина пропусков не связана с отсутствием работы у клиентов, т.к. большинство из них трудоустроены.<br><br>
Попробуем сравнить с ежемесячным доходом.

In [19]:
# Посомтрим клиенты с каким уровнем дохода попали в отфильтрованную таблицу
data_days_empty['total_income'].value_counts()

Series([], Name: total_income, dtype: int64)

Мы получили пустую колонку. Таким образом, у нас имеется 2 колонки, с пропусками в значениях для одних и тех же клиентов.<br>
Проверим обратную ситуацию - для всех ли клиентов с отсутствующим уровнем дохода будет отсутствовать и стаж работы?

In [20]:
# Клиенты с пропусками в колонке 'total_income'
data[data['total_income'].isna()]['days_employed'].value_counts()

Series([], Name: days_employed, dtype: int64)

Наша гипотеза подтвердилась: в обеих колонках с пропусками данные пропущены для одних и тех же клиентов.<br>
Вероятнее всего, данные не были подгружены по техническим причинам и скорее всего эти данные брались из другого источника, что объясняло бы одинаковое количество пропусков в обеих колонках.

Чтобы понять можем ли мы не учитывать данные с пропусками в дальнейшем анализе стоит для начала узнать, какую долю они составляют от общего количества исходной таблицы.

In [21]:
# Посчитаем общее количество строк в исходной таблице
data_rows_count = data.shape[0]

# Посчитаем количество незаполненных строк в колонке 'days_employed'
employed_empty_sum = data['days_employed'].isna().sum()
print('Количество строк в таблице: ' + str(data_rows_count) + ', из них пропущено в колонке `days_employed`: ' + str(employed_empty_sum))

# Посчитаем долю незаполненных строк по отношению к общему количеству
employed_empty_share = employed_empty_sum / data_rows_count
print('Доля пропусков в колонке `days_employed` составляет {:.0%}'.format(employed_empty_share))

Количество строк в таблице: 21525, из них пропущено в колонке `days_employed`: 2174
Доля пропусков в колонке `days_employed` составляет 10%


Доля пропущенных значений составляет 10%, что достаточно много и мы рискуем потерять большое количество важной информации в ходе анализа при исключении из анализа этих данных.<br><br>
В таком случае лучше будет заменить пропущенные значения на какое-то общее значение, характерное для всей выборки (либо для конкретной группы), которое мы можем рассчитать с помощью средней или медианы.<br><br>

Прежде чем искать общий показатель для пропущенных значений необходимо разобраться с отрицательными и положительными значениями в колонке `days_employed`

In [22]:
# Выведем информацию о клиентах с отрицательным значением стажа работы
data[data['days_employed'] < 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи
10,2,-4171.483647,36,высшее,0,женат / замужем,0,M,компаньон,0,113943.491460,покупка недвижимости


В случае с отрицательными значениями можно предположить, что в колонке действительно указаны дни. Если обратиться к сторонним источникам на тему того, как рассчитывается стаж работы, то можно выяснить, что отрицательный стаж работы может возникать в случае, если клиент до сих пор трудоустроен.<br>
Сгруппируем данные с отрицательным стажем по колонке `income_type` и посмотрим окажутся ли в ней такие категории, как <i>"пенсионеры"</i> и <i>"безработные"</i>.

In [23]:
# Выведем информацию о клиентах с отрицательным значением стажа работы и сгруппируем ее по колонке 'income_type'
(
    data[data['days_employed'] < 0]
    .groupby('income_type')
    .agg({
        'days_employed': ['min','mean','median','max']
    })
)

days_employed                                       
                          min         mean       median          max
income_type                                                         
в декрете        -3296.759962 -3296.759962 -3296.759962 -3296.759962
госслужащий     -15193.032201 -3399.896902 -2689.368353   -39.954170
компаньон       -17615.563266 -2111.524398 -1547.382223   -30.195337
предприниматель   -520.848083  -520.848083  -520.848083  -520.848083
сотрудник       -18388.949901 -2326.499216 -1574.202821   -24.141633
студент           -578.751554  -578.751554  -578.751554  -578.751554

Действительно, отрицательные значения стажа работы относятся к тем клиентам, которые на данный момент трудоустроены.<br><br>
Изучим данные по стажу работы с положительными значениями и так же сгруппируем их по колонке `income_type`.

In [24]:
# Выведем информацию о клиентах с положительным значением стажа работы
data[data['days_employed'] > 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
35,0,394021.072184,68,среднее,1,гражданский брак,1,M,пенсионер,0,77805.677436,на проведение свадьбы
50,0,353731.432338,63,среднее,1,женат / замужем,0,F,пенсионер,0,92342.730612,автомобили
56,0,370145.087237,64,среднее,1,вдовец / вдова,2,F,пенсионер,0,149141.043533,образование
71,0,338113.529892,62,среднее,1,женат / замужем,0,F,пенсионер,0,43929.696397,автомобили
78,0,359722.945074,61,высшее,0,женат / замужем,0,M,пенсионер,0,175127.646000,сделка с автомобилем


In [25]:
# Выведем информацию о клиентах с положительным значением стажа работы и сгруппируем ее по колонке 'income_type'
(
    data[data['days_employed'] > 0]
    .groupby('income_type')
    .agg({
        'days_employed': ['min','mean','median','max']
    })
)

days_employed                                             
                       min           mean         median            max
income_type                                                            
безработный  337524.466835  366413.652744  366413.652744  395302.838654
пенсионер    328728.720605  365003.491245  365213.306266  401755.400475

В случае с положительными значениями не выглядит будто была передана информация в днях.<br><br>
Предположим, что информация передается в днях и попробуем перевести их в годы, поделив на 365 дней в году.

In [26]:
# Посчитаем количество лет стажа работы для положительных значений колонки 'days_employed'
# Возьмем первые 10 значений
for days in list(data[data['days_employed'] > 0][0:10]['days_employed']):
    print(int(days / 365))

932
1096
927
996
919
1079
969
1014
926
985


Получаем клиентов со стажем работы около 1000 лет.<br>
Возможно, по ошибке, мы получили данные по пенсионерам и безработным не в днях, а в часах.<br><br>
Попробуем теперь перевести значения колонки `income_type` из часов в дни, а потом в годы, поделив на 24 часа и на 365 дней в году.

In [27]:
# Посчитаем количество лет стажа работы для положительных значений колонки 'days_employed', 
# предварительно переведя их из часов в дни
# Возьмем первые 10 значений
for days in list(data[data['days_employed'] > 0][0:10]['days_employed']):
    print(int(days / 24 / 365))

38
45
38
41
38
44
40
42
38
41


Версия со значениями переданными в часах выглядит правдоподобной.<br><br>
Таким образом, проделаем 2 операции:
- Переведем положительные значения колонки `days_employed` в дни, поделив их на 24 часа.
- Переведем отрицательные значения колонки `days_employed` в положительные, умножив их на -1.

Создадим для этого функцию, которую применим к колонке `days_employed`

In [28]:
# Создадим функцию для форматирования стажа работы клиента
def days_employed_correction(days):
    if days < 0:
        return days * -1
    if days > 0:
        return days / 24
    return days

In [29]:
# Применим функцию 'days_employed_correction' и перезапишем значения в колонку 'days_employed'
data['days_employed'] = data['days_employed'].apply(days_employed_correction)

Построим таблицу, сгруппировав ее по колонке `income_type`.<br>
Сразу выведем агргеацию по минимальному, среднему, медиане и максимальному значениям.

In [30]:
# Сгруппируем данные по колонкам 'income_type' и рассчитаем показатели min, mean, median, max для колонки 'days_employed'
days_employed_grouped_by_income_type = (
    data.groupby('income_type')
    .agg({
        'days_employed': ['min','mean','median','max']
    })
)
days_employed_grouped_by_income_type

days_employed                                          
                          min          mean        median           max
income_type                                                            
безработный      14063.519451  15267.235531  15267.235531  16470.951611
в декрете         3296.759962   3296.759962   3296.759962   3296.759962
госслужащий         39.954170   3399.896902   2689.368353  15193.032201
компаньон           30.195337   2111.524398   1547.382223  17615.563266
пенсионер        13697.030025  15208.478802  15217.221094  16739.808353
предприниматель    520.848083    520.848083    520.848083    520.848083
сотрудник           24.141633   2326.499216   1574.202821  18388.949901
студент            578.751554    578.751554    578.751554    578.751554

In [31]:
# Сохраним значения средних для каждой из группы колонки 'income_type'
days_employed_grouped_by_income_type_mean = (
    data.groupby('income_type')
    .agg({'days_employed': ['mean']})
)

days_employed_grouped_by_income_type_mean = (
    days_employed_grouped_by_income_type_mean
    .reset_index()
)

days_employed_grouped_by_income_type_mean.columns = ['income_type', 
                                                     'days_employed_mean']

days_employed_grouped_by_income_type_mean

,income_type,days_employed_mean
0,безработный,15267.235531
1,в декрете,3296.759962
2,госслужащий,3399.896902
3,компаньон,2111.524398
4,пенсионер,15208.478802
5,предприниматель,520.848083
6,сотрудник,2326.499216
7,студент,578.751554


Мы избавились от отрицательных значений и перевели значения из часов в дни.<br>
Полученную таблицу мы можем использовать для заполнения пропусков в колонке `days_employed`. Т.к. стаж может сильно различаться для всей выборки, то целесообразным будет найти средние значения по группам, например, взяв средний показатель для каждой из групп `income_type`.

In [32]:
# Объединим исходную таблицу и таблицу со средним значением по группам 'income_type'
data = data.merge(
    days_employed_grouped_by_income_type_mean, 
    on="income_type", 
    how="left")

# Заменим пропущенные значения в колонке 'days_employed' на средние значения
data['days_employed'] = data['days_employed'].fillna(data['days_employed_mean'])

# Проверим остались ли пропуски
data[data['days_employed'].isna()].count()

children              0
days_employed         0
dob_years             0
education             0
education_id          0
family_status         0
family_status_id      0
gender                0
income_type           0
debt                  0
total_income          0
purpose               0
days_employed_mean    0
dtype: int64

Мы избавились от пропущенных значений в колонке `days_employed`.<br>
Проделаем такую же операцию для колонки с доходом - `total_income`.<br>
Найдем средние значения для групп `income_type`, объединим таблицы и заменим пропуски на средние.

In [33]:
# Создадим таблицу со средними значениями 'total_income' для каждой группы 'income_type'
total_income_grouped_by_income_type_mean = (
    data.groupby('income_type')
    .agg({'total_income': ['mean']})
)

total_income_grouped_by_income_type_mean = (
    total_income_grouped_by_income_type_mean
    .reset_index()
)

total_income_grouped_by_income_type_mean.columns = ['income_type', 'income_mean']

# Объединим исходную таблицу и таблицу со средним значением по группам 'income_type'
data = data.merge(
    total_income_grouped_by_income_type_mean, 
    on="income_type", 
    how="left")

# Заменим пропущенные значения в колонке 'total_income' на средние значения
data['total_income'] = data['total_income'].fillna(data['income_mean'])

# Проверим остались ли пропуски
data[data['total_income'].isna()].count()

children              0
days_employed         0
dob_years             0
education             0
education_id          0
family_status         0
family_status_id      0
gender                0
income_type           0
debt                  0
total_income          0
purpose               0
days_employed_mean    0
income_mean           0
dtype: int64

Мы избавились от пропущенных значений в колонке `total_income`.<br>
Можно удалить из таблицы колонки `(days_employed, mean)` и `(total_income, mean)` при помощи метода `DataFrame.drop()`.

In [34]:
data = data.drop(columns=['days_employed_mean',
                          'income_mean'])
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,14330.725172,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


**Вывод**

Были найдены пропуски в колонках `days_employed` и `total_income`. Для каждой из них были найдены средние значения для каждой группы `income_type`, которыми мы заменили пропуски в колонках.<br>

Были найдены ошибки при загрузке данных о стаже работы клиентов: для клиентов, которые до сих пор трудоустроены значения были отрицательными и в днях, а для нетрудоустроенных клиентов значения были в часах. Убедившись в этом значения были приведены к единому формату.

### Замена типа данных

В исходных данных есть колнки `days_employed` и `total_income` с вещественным типом данных, который для удобства лучше перевести в целочисленный.<br>
Для перевода в целочисленный тип используем метод `astype()`, т.к. в отличие от метода `to_numeric()` он позволяет переводить значения в целочисленный тип данных.

In [35]:
# Перевеодим значения из вещественного типа в целочисленный
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

При помощи метода `astype()` была произведена замена вещественного типа данных на целочисленный в колонках `days_employed` и `total_income`.

### Обработка дубликатов

В колонке `education` в таблице имеются значения о полученном клиентом образовании. Некоторые из значений одинаковые по смыслу, но находятся в разном регистре. Без обработки данной колонки мы можем упустить неявные дубли.<br>
Чтобы избежать таких неудобств можно выделить данные категории в отдельный словарь, который можно будет связать по идентификатору категории образования, который уже имеется в исходных данных - колонка `education_id`.

Переведем все значения колонки `education` в нижний регистр. После этого избавимся от всех дублей и отсортируем колонку `education_id` в порядке возрастания.

In [36]:
# Переведем в нижний регистр значения в колонке 'education'
data['education'] = data['education'].str.lower()

# Выведем количество значений в колонке 'education'
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверим наши данные на наличие дубликатов.<br>
Для этого применим метод `duplicated()`, который вернет значение `True`, если будут найдены дубли строк.

In [37]:
# Выведем информацию о дублях и их количестве
data.duplicated().value_counts()

False    21454
True        71
dtype: int64

В данных имеется 54 строки с дублями. Выведем их.

In [38]:
# Выведем таблицу с дублями
data[data.duplicated() == True]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2326,41,среднее,1,женат / замужем,0,F,сотрудник,0,161380,покупка жилья для семьи
3290,0,15208,58,среднее,1,гражданский брак,1,F,пенсионер,0,137127,сыграть свадьбу
4182,1,2326,34,высшее,0,гражданский брак,1,F,сотрудник,0,161380,свадьба
4851,0,15208,60,среднее,1,гражданский брак,1,F,пенсионер,0,137127,свадьба
5557,0,15208,58,среднее,1,гражданский брак,1,F,пенсионер,0,137127,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,15208,64,среднее,1,женат / замужем,0,F,пенсионер,0,137127,дополнительное образование
21032,0,15208,60,среднее,1,женат / замужем,0,F,пенсионер,0,137127,заняться образованием
21132,0,2326,47,среднее,1,женат / замужем,0,F,сотрудник,0,161380,ремонт жилью
21281,1,2326,30,высшее,0,женат / замужем,0,F,сотрудник,0,161380,покупка коммерческой недвижимости


Мы получили список дублей.<br><br>
Исходя из данных у нас недостаточно информации, чтобы быть уверенными в том, что полученная информация является действительно дублем. В данных отсутствуют какие-либо уникальные идентификаторы клиентов.<br>

Уникальными могли являться данные со стажем работы и месячным доходом, но в случаях пропусков мы их заменили на средние значения, что в результате и могло стать одной из причин возникновения дублей.<br>

Полноценной взможности убедиться в уникальности этих строк у нас нет, а сама таблица с дублями достаточно мала, чтобы исказить итоговые результаты, при их исключении из анализа.<br><br>

Очистим данные от полученных дублей при помощи метода `drop_duplicates()`.

In [39]:
# Очищаем данные от дублей
data = data.drop_duplicates().reset_index(drop=True)

# Проверяем остались ли дубли
data[data.duplicated() == True].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Данные очищены от дублей.

**Вывод**

Были выявлены дубли в данных. Полной уверенности в том, что это были дубли, а не разные клиенты со схожими характеристиками, у нас нет.<br>
Вероятно, что к возникновению дублей привели следующие моменты:

- отсутствие идентификатора клиента;
- замена пропусков на средние показатели стажа работы клиента и месячного дохода.

Было решено очистить данные от полученных дублей, учитывая, что их количество было не большим.

### Лемматизация

В колонке `purpose` указана цель полчения кредита клиентом. Среди перечисленных целей есть похожие по смыслу, но по-разному сформулированные - такие цели можно сгруппировать по категориям.<br>
Можно выделить следующие категории:
- недвижимость
- автомобиль
- свадьба
- образование

Для категоризации целей используем лемматизацию при помощи библиотеки `pymystem3`.<br>
Импортируем библиотеку.

In [43]:
# Импорт библиотеки pymystem3 для лемматизации
from pymystem3 import Mystem
m = Mystem()

Создадим словарь, в котором ключами будут названия категорий, а значениями будут ключевые слова, по вхождению которых будем определять принадлежность к той или иной категории.

In [44]:
# Создадим словарь для лемматизации, в котором ключом будет название категории цели получения кредита, 
# а в значении будут ключевые слова для группировки по категориям
purpose_dict = {'недвижимость': ['недвижимость', 'жилье'],
                'автомобиль': ['автомобиль'],
                'свадьба': ['свадьба'],
                'образование': ['образование']}

Создадим функцию для определения категории по переданной строке с целью кредита.<br>

Функция будет принимать строку с указанной целью получения кредита, эта строка будет переведена в леммы, т.е. к его словарной форме.<br>

Далее функция будет проходиться по каждому ключу словаря `purpose_dict` и по каждому значению (или значениям) каждого ключа словаря, и при нахождении одного из значений ключа, функция будет возвращать ключ в качестве названия подходящей категории. К функции добавим проверку ошибок, при помощи `try-except`, на случай если совпадений по ключам не найдется.

In [45]:
# Создадим функцию для определения категории цели получения кредита
def purpose_category(string):
    lemmatize_string = m.lemmatize(string)
    for purposes in purpose_dict:
        for purpose in purpose_dict[purposes]:
            lemmatize_purpose = m.lemmatize(purpose)
            try:
                if lemmatize_purpose[0] in lemmatize_string:
                    return purposes
            except:
                return 'unknown'

Создадим новую колонку `purpose_category` и применим к ней созданную функцию `purpose_category` для заполнения значениями категорий при помощи метода `apply()`.

In [46]:
# Применим функцию 'purpose_category' и сохраним полученные значения в новую колонку 'purpose_category'
data['purpose_category'] = data['purpose'].apply(purpose_category)

# Выведем количество значений в колонке 'purpose_category'
data['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

Проверим количество значений в обеих колонках при помощи метода `info()`.

In [47]:
# Выведем информацию о колонках 'purpose' и 'purpose_category' 
data[['purpose','purpose_category']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   purpose           21454 non-null  object
 1   purpose_category  21454 non-null  object
dtypes: object(2)
memory usage: 335.3+ KB


Количество заполненных значений совпадает и пропуски отсутствуют.

**Вывод**

Было создано 4 категории для определения цели получения кредита:
- недвижимость
- автомобиль
- свадьба
- образование

Была создана отдельная колонка `purpose_category`, в которую при помощи лемматизации были сохранены значения категорий.

### Категоризация данных

Для проведения дальнейшего анализа необходимо провести категоризацию данных, чтобы избежать следующих неудобств и проблем в ходе анализа:
- Усложнение визуальной работы с таблицей;
- Увеличение размера файла и скорости обработки данных;
- Сложность в фильтрации данных, если необходимо фильтровать по длинным названиям (увеличивается вероятность допустить ошибку при вводе);
- Сложнее создавать новые категории и изменять старые;
- Сложнее делать статистические выводы со значениями которые встречаются единожды (например, ежемесячный доход).

В исходных данных у нас имеется 2 колонки, для которых в таблице выведены еще и колонки с `id` - это колонка `education` и `family_status`.<br>
Для этих колонок можно провести категоризацию данных.

Выведем таблицу по колонкам `education` и `education_id` и сохраним ее в отдельную переменную `education_dict`.

In [48]:
# Сохранение в переменной 'education_dict' таблицы с колонками 'education_id' и 'education'
education_dict = data[['education_id', 'education']]

# Выведем количество значений в колонке 'education'
education_dict['education'].value_counts()

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [49]:
# Удалим все дубликаты в таблице 'education_dict'
education_dict = (
    education_dict.drop_duplicates()
    .reset_index(drop=True)
    .sort_values('education_id')
)

# Выведем таблицу 'education_dict'
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Мы создали "словарь" `education_dict`, в котором хранится информация о видах образования.<br>
Можно удалить колонку `education` в исходной таблице с помощью метода DataFrame.drop()

In [50]:
# Удалим колонку 'education' из таблицы 'data'
data = data.drop(columns=['education'])
data

,children,days_employed,dob_years,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость
21450,0,14330,67,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль
21451,1,2113,38,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость
21452,3,3112,38,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль


Создадим словарь для колонки `family_status`.<br>
Выведем таблицу с колонками `family_status_id` и `family_status` и сохраним ее в пременную `family_status_dict`.<br>
Проведем для нее те же операции, как и для колонки с видом образования:
- переведем значения в нижний регистр;
- избавимся от дубликатов;
- отсортируем значения по колонке с идентификатором;
- удалим колонку `family_status` из исходной таблицы.

In [52]:
family_status_dict = (
    data[['family_status_id', 'family_status']].copy()
)

family_status_dict['family_status'] = (
    family_status_dict['family_status'].str.lower()
)

family_status_dict = (
    family_status_dict.drop_duplicates()
    .reset_index(drop=True)
    .sort_values('family_status_id')
)

family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [53]:
# Удалим колонку 'family_status' из таблицы 'data'
data = data.drop(columns=['family_status'])
data

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем,недвижимость
21450,0,14330,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль
21451,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость,недвижимость
21452,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль


В колонке `total_income` в таблице имеются разные значения дохода. Чтобы провести анализ и увидеть какую-либо закономерность нам необходимо сгруппировать значения по категориям.

Посмотрим для начала диапазон значений. Выведем минимальное и максимальное значение.

In [54]:
# Выведем миинимальное значение колонки 'total_income'
data['total_income'].min()

20667

In [55]:
# Выведем максимальное значение колонки 'total_income'
data['total_income'].max()

2265604

Разброс значений находится в диапазоне от 20000 до 2 300 000. Создадим функцию, которая будет определять категорию по значению колонки `total_income`. Разбивать будем на 5 категорий:
- `< 500 000`;
- `500 000 - 1 500 000`;
- `> 1 500 000`

In [56]:
# Создадим функцию для определения группы по ежемесячному дохооду
def group_income(income):
    if income < 500000:
        return '1. < 500 000'
    if income < 1500000:
        return '2. > 1 500 000'
    return '3. от 1 500 000 и более'

In [57]:
# Применим функцию 'group_income' к колонке 'total_income' и сохраним полученные значения в колонку 'income_category'
data['income_category'] = data['total_income'].apply(group_income)

# Выведем количество значений в созданной колонке
data['income_category'].value_counts()

1. < 500 000               21232
2. > 1 500 000               215
3. от 1 500 000 и более        7
Name: income_category, dtype: int64

Проведем подобную категоризацию и по количеству детей.

Для начала посмотрим какие значения попадаются в колонке `children`.

In [58]:
# Выведем значения в колонке 'children'
data['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Мы видим, что в данных имеются странные значенияЖ -1 и 20. Такие данные выглядят ошибочными, тем более, заметно количество семей с большим числом детей в семье уменьшается, но при этом клиентов с количеством детей 20 очень много.<br>
Чтобы избежать искажения результатов анализа избавимся от таких значений.

In [67]:
# Перезапишем датафрейм, убрав строки с ошибочными значениями в колонке 'children'
data = data[
    (data['children'] >= 0) & (data['children'] < 20)
].copy()

# Выведем значения колонки 'children'
data['children'].value_counts()

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Создадим функцию для разделения на 3 группы по наличию детей: 0, 1-2, 3 и более.

In [68]:
def children_category(children_amount):
    if children_amount == 0:
        return '0'
    if children_amount == 1 or children_amount == 2:
        return '1-2'
    return '3 и более'

Создадим колонку `children_category` и применим к ней функцию

In [69]:
data['children_category'] = data['children'].apply(children_category)

data['children_category'].value_counts()

0            14091
1-2           6860
3 и более      380
Name: children_category, dtype: int64

**Вывод**

Для удобства было выделено 2 словаря для уровня образования клиентов и их семейного положения.

## 3. Ответы на поставленные в задаче вопросы

#### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Проверим зависимость, построив сводную таблицу с группировками, где в индексе `children`, в колонках - `debt`, и с агргеацией по количеству значений в `gender`, т.к. эта характеристика присутствует у всех клиентов.

In [70]:
# Построим сводную таблицу
children_pivot = (
    data.pivot_table(index='children_category', 
                     columns='debt', 
                     values='gender', 
                     aggfunc='count')
)

children_pivot

debt,0,1
children_category,,
0,13028,1063
1-2,6222,638
3 и более,349,31


У нас получились группы с разными количествами клиентов, чтобы мы могли корректно сравнить корреляцию можду группами необходимо рассчитать доли должников, среди всех представителей группы.

In [73]:
# Посчитаем долю должников по группам
children_pivot['share'] = (
    100 * children_pivot[1] / 
    (children_pivot[0] + children_pivot[1])
)

children_pivot

debt,0,1,share
children_category,,,
0,13028,1063,7.543822
1-2,6222,638,9.300292
3 и более,349,31,8.157895


**Вывод**

Зависимость не сильная, но присутствует: люди, не имеющие детей, возвращают кредиты с большей вероятностью, а в случаях, где дети присутствуют, количество детей увеличивает вероятность невозврата кредита в срок.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Построим сводную таблицу и посчитаем долю должников.

In [74]:
# Построим сводную таблицу
family_status_pivot = (
    data.pivot_table(index='family_status_id', 
                     columns='debt', 
                     values='gender', 
                     aggfunc='count')
)

# Посчитаем долю должников по группам
family_status_pivot['share'] = (
    100 * family_status_pivot[1] / 
    (family_status_pivot[0] + family_status_pivot[1])
)

# Отсортируем полученную таблицу по возрастанию
family_status_pivot.sort_values('share')

debt,0,1,share
family_status_id,,,
2,888,63,6.624606
3,1105,84,7.064760
0,11334,927,7.560558
1,3749,385,9.313014
4,2523,273,9.763948


In [75]:
# Выведем созданный ранее словарь 'family_status_dict'
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


**Вывод**

Результаты сводной таблицы показывают, что показатель возврата кредита в срок выше у клиентов с семейным положением `вдовец / вдова`, далее идут клиенты с семейным положением `в разводе` и `женат / замужем`, а хуже всего выплачивают кредиты те, кто находится в семейном положени `гражданский брак` и `не женат / не замужем`.<br>

Можно сделать вывод, что клиенты, которые еще не завели семью, либо завели в статусе "гражданского брака", менее склонны возвращать кредиты вовремя, в сравнении с клиентами, у которых есть, либо уже была семья. Вероятно это связано с дополнительными обязательствами, в том числе и финансовыми, возникающие во время брака, либо развода.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [76]:
# Построим сводную таблицу
income_pivot = (
    data.pivot_table(index='income_category', 
                     columns='debt', 
                     values='gender', 
                     aggfunc='count')
)

# Посчитаем долю должников по группам
income_pivot['share'] = (
    100 * income_pivot[1] / 
    (income_pivot[0] + income_pivot[1])
)

income_pivot

debt,0,1,share
income_category,,,
1. < 500 000,19391,1718,8.138709
2. > 1 500 000,202,13,6.046512
3. от 1 500 000 и более,6,1,14.285714


Видно, что чем меньше доход, тем больше в группе должников. Так же, видно, что размер группы сильно уменьшается, с увеличением дохода.<br>
Посчитаем и выведем размер каждой группы.

In [77]:
# Посчитаем размер каждой группы
income_pivot['group_count'] = income_pivot[0] + income_pivot[1]
income_pivot

debt,0,1,share,group_count
income_category,,,,
1. < 500 000,19391,1718,8.138709,21109
2. > 1 500 000,202,13,6.046512,215
3. от 1 500 000 и более,6,1,14.285714,7


**Вывод**

Можно сделать вывод, что высокий уровень дохода влияет не только на вероятность вовремя выплачивать кредит, но и на то, что такие люди впринципе реже берут кредиты.<br>

Если учесть размеры выборок, то можно предположить о недостатке данных для групп с большим заработком, в связи с чем результаты по этим группам могут выглядеть более оптимистичными, чем на самом деле.

#### Как разные цели кредита влияют на его возврат в срок?

In [78]:
# Построим сводную таблицу
purpose_pivot = (
    data.pivot_table(index='purpose_category', 
                     columns='debt', 
                     values='gender', 
                     aggfunc='count')
)

# Посчитаем долю должников по группам
purpose_pivot['share'] = (
    100 * purpose_pivot[1] / 
    (purpose_pivot[0] + purpose_pivot[1])
)

purpose_pivot

debt,0,1,share
purpose_category,,,
автомобиль,3879,400,9.347978
недвижимость,9971,780,7.255139
образование,3619,369,9.252758
свадьба,2130,183,7.911803


**Вывод**

Заметна определенная зависимость между целью получения кредита и вероятностью его возврата в срок. Клиенты, взявшие кредиты на покупку автомобиля и  на получение образования в меньшей степени склонны к тому, чтобы возвращать кредит в срок, в отличии от тех, кто брал кредит на покупку недвижимости или свадьбу. Это можно объяснить тем, что в случае свадьбы - деньги могут быть возвращены с полученных подарков, а в случае недвижимости - со сдачи недвижимости в аренду, но а если клиент сам проживает в ней, то у него гораздо больше мотивации выплачивать кредит, чтобы не остаться без жилья. Еще такую взаимосвязь можно объяснить тем, что недвижимость часто берут в кредит семейные пары, как и кредит на свадьбу, и больше вероятность, что совместно они смогут вовремя погасить долги по кредиту.

## 4. Общий вывод

В ходе анализа были найдены следующие взаимосвязи в отношении вероятности возврата кредита в срок. Рассмотрим каждый фактор в отдельности:

1. Наличие детей - является скорее минусом, так как в трудном финансовом положении человек, имеющий детей, предпочтет потратить средства на ребенка.
2. Семейное положение - люди состоящие в официальном браке или состояшие в нем раньше более ответственные.
3. Уровень достатка - люди со средним доходом чаще остальных берут кредиты, однако выплачивают их не очень исправно. В то время как люди с низким достатком имеют меньше просрочек по выплатам. Клиенты с высоким доходом с большей вероятностью выплачивают долги в срок, хоть и гораздо реже берут кредиты в целом.
4. Цель кредита - самая популярная цель это "недвижимость". По ней так же и меньше всего просрочек с выплатами. Можно предположить, что вопрос улучшения желищных условий действительно важен для людей. Клиенты, взявшие кредит на покупку автомобиля и на получение образования в большей степени склонны не выплачивать кредиты в срок.

После изучения данных по данной выборке можно сделать заключение, что все 4 фактора, которые мы рассмотрелии являются важными в оценке рисков при выдаче кредита. Можно утвержать, что чаще всего кредиты берут люди со средним достатком с целью проведения операций с недвижимостью, не состоящие в официальном браке и не имеющие детей. Самыми надежными клиентами являются люди состоящие (ныне или ранее) в браке, без детей, с низким уровнем достатка и целью получения кредита на недвижимость.